# Finger Trees

In [24]:
:ext FlexibleInstances
:ext MultiParamTypeClasses
:ext DeriveGeneric
:ext AutoDeriveTypeable

:m Data.FingerTree
import Data.Monoid hiding ((<>))
:m Data.Semigroup
import Prelude hiding (null)

-- Indexed array (logarithmic access)

newtype Size a = Size { getSize :: a } deriving (Show, Eq)

instance Measured (Sum Int) (Size a) where
  measure _ = Sum 1
  
alphabet :: FingerTree (Sum Int) (Size Char)
alphabet = fromList . map Size . take 26 $ enumFrom 'a'

atIndex :: Int -> FingerTree (Sum Int) (Size a) -> Maybe a
atIndex n t =
  case viewl . snd $ split (> Sum n) t of
    Size c :< _ -> Just c
    _ -> Nothing
    
atIndex 16 alphabet

-- Priority Queue

data Entry k v = Entry k v deriving Show

instance Functor (Entry k) where
    fmap f (Entry k v) = Entry k (f v)

instance Foldable (Entry k) where
    foldMap f (Entry _ v) = f v

data Prio k v = NoPrio | Prio k v deriving Show

instance Ord k => Semigroup (Prio k v) where
  (<>) = unionPrio
  
instance Ord k => Monoid (Prio k v) where
  mempty = NoPrio
  mappend = unionPrio
  
pqUnion :: Ord k => PQueue k v -> PQueue k v -> PQueue k v
pqUnion (PQueue xs) (PQueue ys) = PQueue (xs >< ys)
  
unionPrio :: Ord k => Prio k v -> Prio k v -> Prio k v
x `unionPrio` NoPrio      = x
NoPrio `unionPrio` y      = y
x@(Prio kx _) `unionPrio` y@(Prio ky _)
  | kx <= ky            = x
  | otherwise           = y
  
instance Ord k => Measured (Prio k v) (Entry k v) where
    measure (Entry k v) = Prio k v
  
newtype PQueue k v = PQueue (FingerTree (Prio k v) (Entry k v)) deriving Show

minViewWithKey :: Ord k => PQueue k v -> Maybe ((k, v), PQueue k v)
minViewWithKey (PQueue q)
  | null q = Nothing
  | otherwise = Just ((k, v), case viewl r of
    _ :< r' -> PQueue (l >< r')
    _ -> error "can't happen")
  where
    Prio k v = measure q
    (l, r) = split (below k) q
    
below :: Ord k => k -> Prio k v -> Bool
below _ NoPrio = False
below k (Prio k' _) = k' <= k

insert :: Ord k => k -> v -> PQueue k v -> PQueue k v
insert k v (PQueue q) = PQueue (Entry k v <| q)

pqFromList :: Ord k => [(k, v)] -> PQueue k v
pqFromList = foldr (uncurry insert) (PQueue empty)

queue = pqFromList [(3, 'a'), (1, 'z'), (5, 't'), (3, 'x'), (0, 'm')]

minViewWithKey queue

let (PQueue t) = queue
measure t

Just 'q'

Just ((0,'m'),PQueue (fromList [Entry 3 'a',Entry 1 'z',Entry 5 't',Entry 3 'x']))

Prio 0 'm'